# STA 141B Data & Web Technologies for Data Analysis

### Lecture 19, 12/6/23, PCA

### Announcements

- last homework due tomorrow

### Final report 

- revisit the grading rubric given in lecture 7 
- submit: a document (e.g., pdf) with your report. Include a file (e.g., .R or .RMD) with your runnable code. 
- submit _once per group_! 
- submit _on time_! (late submissions will not be accepted) 
- clearly state each team members contribution in the report. 


### Todays topics

- Principal Component Analysis

### Motivation

We aim to find a method that reduces high-dimensional data to a lower dimension with as little information loss as possible. Consider the problem in regression: 

In [ ]:
adv <- read.csv('../data/Advertising.csv')
adv <- adv[,-1]
adv$Budget <- rowSums(adv[,-4])
fitted1 <- lm(Sales ~ Budget, data = adv)$fitted
fitted2 <- loess(Sales ~ Budget, data = adv)$fitted

    options(repr.plot.width = 20, repr.plot.height = 12)

plt1 <- ggplot(adv, aes(Budget, Sales)) + geom_point() + theme_classic()
plt2 <- ggplot(adv, aes(Budget, Sales)) + theme_classic() + 
        geom_point(alpha = 0.1) + 
        geom_smooth(method = 'lm', se = F) + 
        geom_point(aes(y=fitted1), color = 'blue') + 
        geom_segment(aes(xend = Budget, yend = fitted1), color = 'red', alpha = 0.5)
plt3 <- ggplot(adv, aes(Budget, Sales)) + theme_classic() + 
        geom_point(alpha = 0.1) + 
        geom_smooth(method = 'loess', se = F) + 
        geom_point(aes(y=fitted2), color = 'blue') + 
        geom_segment(aes(xend = Budget, yend = fitted2), color = 'red', alpha = 0.5)

require("ggplot2")

In [ ]:
gridExtra::grid.arrange(plt1, plt2, plt3,ncol = 3)

### Principal Component Analysis 

Consider a data set $X\in\mathbb{R}^{n\times p}$ for $n, p\in\mathbb{N}$. 
Each observation corresponds to a random vector $x = (x_1, \dots , x_p)'$  with known expectation $E(x) = \mu\in\mathbb{R}^p$ and covariance $Cov(x) = \Sigma = \mathbb{R}^{p\times p}$. 

Its principal components are linear combinations of $x_1, \dots , x_p$. Specifically, we aim to find a tranformation of $x$ so that the covarince of $U(x-\mu)$, $U\in\mathbb{R}^{p\times k}$, $k\leq p$ has a simple structure and retains as much information about $\Sigma$ as possible. 

Let $u_1\in\mathbb{R}^p$ so that $Cov(u_1'(x-\mu)) =  Cov(u_1'x) = u_1'\Sigma u_1$ is maximal. 
Since $u_1$ can be arbitrarily large, we limit $\|u_1\|^2 = 1$. 

To maximize $u_1'\Sigma u_1$ under this constraint, one can use Lagrange multiplication for $\lambda>1$.  
$$
\max u_1'\Sigma u_1 - \lambda(\|u_1\|^2 - 1). 
$$
Taking derivatives gives $(\Sigma - \lambda I_p) u_1 = 0$. Hence, $u_1$ is an eigenvector to eigenvalue $\lambda$ of $\Sigma$. 
Since $u_1'\Sigma u_1$ ought to be maximized, $\lambda$ should be as large as possible. 
Consequently, $u_1$ is an eigenvector to the largest eigenvalue $\lambda$ of $\Sigma$. 
The vector $u_1'(x-\mu)$ is the first *principal component* of $x$.

Generally, one can show that the $k$-th principal component is $u_k'(x-\mu)$ is the eigenvector corresponding to the $k$-th eigenvalue of $\Sigma$. 
The vector $u_k\in\mathbb{R}^p$ is the vectors of loadings of the $k$-th principal component. 

For $U\in\mathbb{R}^{p\times k}$ be an orthogonal matrix, which $k$-th column is $u_k$ and $z = U'(x − \mu)\in\mathbb{R}^k$, $k \leq p$ the vector, which $k$-th element is the $k$-th principal component. 

Note that for $k=p$ we have $$\Sigma = U\text{diag}(\lambda_1, \dots, \lambda_p)U'$$ and $$Cov(z) = E(zz') = U'E((x-\mu)(x-\mu)')U = U'\Sigma U = \text{diag}(\lambda_1, \dots, \lambda_p).$$ 

In practive, $\mu$ and $\Sigma$ are unkown. For the data set $X\in\mathbb{R}^{n\times p}$, the empirical principal components are given from the eigenvalue decomposition $\widehat\Sigma = \widehat U\text{diag}(\lambda_1, \dots, \lambda_p)\widehat U'$. 
If $n>p$ one may use the consistent estimators 
$$
\widehat\Sigma = \frac{1}{n - 1}\widetilde{X}'\widetilde{X}, 
$$
where $\widetilde{X}$ is the matrix with entries $\widetilde{X}_{ij} = x_{ij} - \bar{x}_j$, $\bar{x}_j = n^{-1}\sum_{i=1}^n x_{ij}$ is the estimator for $\mu_j$. 
This gives the scores $Z = \widetilde X \widehat{U}\in\mathbb{R}^{n\times p}$. 

Example: Lets consider the case with $p = 2$: 

In [ ]:
adv <- read.csv('../data/Advertising.csv')
n <- nrow(adv)
nrow(adv)

In [ ]:
adv <- adv[,-1]
adv$Budget <- rowSums(adv[,-4])
head(adv)

For graphical issues only, lets standardsize the columns. 

In [ ]:
normalize <- function(x) (x - min(x)) / (max(x) - min(x))

In [ ]:
X <- as.data.frame(apply(adv, 2, normalize))[4:5]
head(X)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)
require('ggplot2')
plt <- ggplot(X, aes(Budget, Sales)) + theme_classic() + 
    xlim(c(-0.4, 1)) + ylim(c(-0.4, 1)) +
    geom_point() + coord_fixed() 
plt + stat_ellipse()

We know that the density of $x$ is constant on each given contour line, i.e. 
$$
c^2 = x'\Sigma^{-1} x. 
$$
After eigenvalue decomposition, 
$$
c^2 = x'U\text{diag}(\lambda_1^{-1},\lambda_2^{-1})U'x = \frac{z_1^2}{\lambda_1} + \frac{z_2^2}{\lambda_2}. 
$$
This equation defines an ellipsoid in the coordinate system $(z_1, z_2)$. As $\lambda_1>\lambda_2$, the main axis of this ellipsoid lies in direction of $z_1$. 

In [ ]:
xbar <- colMeans(X)
Xtilde <- t(t(X) - xbar)

In [ ]:
colMeans(Xtilde)

In [ ]:
Sigmahat <- t(Xtilde) %*% Xtilde / (n - 1)
EIG <- eigen(Sigmahat)

In [ ]:
EIG

In [ ]:
lambda <- EIG$values
Uhat <- EIG$vectors
Z <- Xtilde %*% Uhat

Recall: Eigenvectors are orthogonal to each other. Here, they are also normalized (orthonormal). 

In [ ]:
t(Uhat) %*% Uhat

In [ ]:
head(Z)

Consider the first principal component. 

In [ ]:
slope <- Uhat[2, 1] / Uhat[1, 1]
intercept <- 0
 
plt + geom_abline(intercept = intercept, slope = slope, 
                  color = 'blue', linewidth = 1)

In [ ]:
x1 <- (X[, 2] - intercept) / slope
y1 <- intercept + slope * X[, 1]
x2 <- (x1 + X[, 1]) / 2
y2 <- (y1 + X[, 2]) / 2

df <- data.frame(X,x2,y2)

In [ ]:
plt + geom_segment(aes(xend=x2,yend=y2),color="red", alpha = 0.5)+
      geom_abline(intercept=0,slope=slope, color = 'blue', linewidth = 1)

Second principal component. 

In [ ]:
slope <- Uhat[2, 2] / Uhat[1, 2]
intercept <- 0 

x1 <- (X[, 1] - intercept) / slope
y1 <- intercept + slope * X[, 2]
x2 <- (x1 + X[, 2]) / 2
y2 <- (y1 + X[, 1]) / 2

df <- data.frame(X,x2,y2)
plt + geom_segment(aes(xend=x2,yend=y2),color="red", alpha = 0.5)+
      geom_abline(intercept=0,slope=slope, color = 'blue', linewidth = 1) 

In [ ]:
lambda / sum(lambda)

The data under the new coordinate system: 

In [ ]:
ggplot(as.data.frame(Z)) + theme_classic() + geom_point(aes(V1, V2)) + labs(x = "PC1", y = "-PC2")

### Choice of $k$

We want to use PCA as a dimension reduction. The $p$-dimensinal data is to be reduced to some $k$-dimensional data, $k\le p$, while preserving as much variation (information) as possible. Since

$$
Cov(z) = \text{diag}(\lambda_1, \dots, \lambda_p), 
$$

this corresponds on choosing the first $k$ empirical components. But how to choose $k$? 

Consider the [Beans data set](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset). 

In [ ]:
data <- read.table('../data/Dry_Bean_Dataset.csv', sep = ',', header = T)
data <- na.omit(data)[,-1]
n <- nrow(data)
dim(data)

In [ ]:
head(data)

In [ ]:
X <- data
xbar <- colMeans(X)
Xtilde <- t(t(X) - xbar)
Sigmahat <- t(Xtilde) %*% Xtilde / (n - 1)
EIG <- eigen(Sigmahat)
lambda <- EIG$values
Uhat <- EIG$vectors

In [ ]:
lambda / sum(lambda)

In [ ]:
eigenvalues <- data.frame(index = 1:length(lambda), 
                          relative_variability = lambda / sum(lambda))

In [ ]:
require("ggplot2")
ggplot(eigenvalues) + theme_minimal() + 
    labs(x = 'Eigenvalue', y = 'Relative Variability') + 
    geom_line(aes(index, cumsum(relative_variability))) + 
    geom_point(aes(index, cumsum(relative_variability))) 

The ellbow plot shows the relative variance - the number of PC to be chosen corresponds to the location in the plot, in which the additional variance explained per PC decreases. This is a subjective measure.  

### Computation of Eigenvalues

The computation of eigenvalues is complicated if $p$ is large. Here, we will learn about an approximate way to compute the $k$ largest eigenvectors and -values.  

Let $A\in\mathbb{R}^{p\times p}$ be a symmetric positiv semi-definite matrix, so that $A=V\mbox{diag}(\mu_1,\ldots,\mu_p)V^t$. 
The columns of $V=(v_1,\ldots,v_p)$ are the eigenvectors $v_i$ and $\mu_1>\mu_2>\ldots>\mu_p\geq 0$. To estimate $v_1$, start with any random vector $v^{(0)}$ and iterate 
$$
v^{(j+1)}_1=\frac{A^jv^{(0)}}{\|A^jv^{(0)}\|},\;\;j=1,2,\ldots
$$
until convergence. The corresponding eigenvalue is due to the Rayleigh ratio
$$
\mu_1=\frac{v_1^tAv_1}{v_1^tv_1},
$$
since 
$$
\frac{v_1^tAv_1}{v_1^tv_1}=\frac{(Av_1)^tv_1}{v_1^tv_1}=\frac{\mu_1 v_1^tv_1}{v_1^tv_1}=\mu_1.
$$
The second eigenvector can be retrieved via $A-\mu_1v_1v_1^t$ (there are other methods as well).

$$
v^{(j+1)}_1=\frac{A^jv^{(0)}}{\|A^jv^{(0)}\|},\;\;j=1,2,\ldots
$$

In [ ]:
A <- Sigmahat

In [ ]:
approx_evec <- function(A) {
    v <- runif(nrow(A))
    for (i in 1:10) {
        v <- A %*% v
        v <- v / sqrt(sum(v^2))
    }
    return (drop(c(v)))
}

In [ ]:
a <- approx_evec(A)
a

In [ ]:
Uhat[,1]

Note that the eigenvector is identical up to sign. 

In [ ]:
sum((Uhat[,1] - (- a))^2)

The corresponding eigenvalue is given: 

In [ ]:
mu1 <- c(t(a) %*% A %*% a / t(a) %*% a)
mu1

In [ ]:
lambda[1]

The second eigenvector is given by: 

In [ ]:
b <- approx_evec(A - mu1 * a %*% t(a))
b

In [ ]:
Uhat[,2]

$$
v^{(k+1)}_1=\frac{A^kv^{(0)}}{\|A^kv^{(0)}\|},\;\;k=1,2,\ldots
$$

Why does this work? Since the eigenvectors are orthogonal, each vector $c\in\mathbb{R}^p$ can be represented as a linear combination of eigenvectors, i.e., $c=\sum_{i=1}^p c_iv_i$, $c_i\geq 0$ with $c_1>0$. Multiplicatoin on both sides with $A$ gives 
$$
Ac=\sum_{i=1}^pc_iAv_i=\sum_{i=1}^pc_i\mu_iv_i.
$$
$k$-times repetition gives 
$$
A^kc=\sum_{i=1}^pc_i\mu_i^kv_i=\mu_1^k\left\{c_1v_1+c_2v_2\left(\frac{\mu_2}{\mu_1}\right)^k+\ldots+c_pv_p\left(\frac{\mu_p}{\mu_1}\right)^k\right\}.
$$
For $\mu_1\gg\mu_2$, $A^kc$ converges to $v_1(c_1\mu_1^k)$ for growing $k$. This convergence can be very slow if $\mu_1\approx\mu_2$. 

How to interpret PC? Suppose `Newspaper` and `Radio` are collinear. Then, we can isolate their effect via a PCA. 

In [ ]:
head(adv)

In [ ]:
GGally::ggpairs(adv, columns = 1:3) 

In [ ]:
summary(lm(Sales~. - Budget, adv))

In [ ]:
X <- adv[,2:4]
xbar <- colMeans(X)
Xtilde <- t(t(X) - xbar)
Sigmahat <- t(Xtilde) %*% Xtilde / (n - 1)
EIG <- eigen(Sigmahat)
lambda <- EIG$values
Uhat <- EIG$vectors
Z <- Xtilde %*% Uhat

In [ ]:
head(Z)

In [ ]:
summary(lm(Sales~TV + Z[,1]+ Z[,2], adv))

The first PC emulates the effect of `Newspaper` and `Radio`.

### Downsides of PCA: 
 - If more than one PC are included, their individual effect cannot be related to a real life variable, and hence its more difficult to interpret it
 - PCA can not reduce the dimensionality of observations on a low-dimensinal non-linear manifold